This demo is from demo 6 in [1].

In [1]:
from sympy import symbols
from SOSPy import *
import time

## MAX CUT

MAX CUT is the problem of partitioning nodes in a graph into two disjoint sets $V_1$ and $V_2$, such that the weighted number of nodes that have an endpoint in $V_1$ and the other in $V_2$ is maximuized. This can be formulated as a boolean optimization problem

\begin{gather}
    \max_{x_i\in\{-1,1\}} \frac{1}{2} \sum_{i,j}\,w_{i,j}(1-x_ix_j),
\end{gather}

or equivalently as a constrained optimization

\begin{gather*}
    \max_{x_i^2=1}f(x) \triangleq \max_{x_i^2=1} \frac{1}{2} \sum_{i,j}\,w_{i,j}(1-x_ix_j)
\end{gather*}

Here $w_{ij}$ is the weight of edge connecting nodes $i$ and $j$. For example we can take $w_{ij}=0$ if nodes $i$ and $j$ are not connected, and $w_{ij}=1$ if they are connected. If node $i$ belongs to $V_1$, then $x_i=1$, and conversely $x_i=-1$ if node $i$ is in $V_2$. For example, for connected nodes $x_1, x_2$ and weight $w_{1,2}=1$, $x_1$ is in $V_1$ and $x_2$ is in $V_2$. Then they contribute a weight of $\frac{1}{2} \times 1 \times (1-(-1)) = 1$.

A sufficient condition for $\max_{x_i^2=1}f(x) \leq \gamma$ is as follows. Assume that our graph contains $n$ nodes. Given $f(x)$ and $\gamma$, then $\max_{x_i^2=1}f(x) \leq \gamma$ if there exist sum of squares $p_1(x)$ and polynomials $p_2(x),\dots,p_{n+1}(x)$ such that

\begin{gather*}
    p_1(x)(\gamma-f(x)) + \sum^n_{i=1}(p_{i+1}(x)(x_i^2-1))-(\gamma-f(x))^2 \geq 0 \tag{2}
\end{gather*}

This can be proved by a contradiction. Suppose there exists $x\in {-1,1}^n$ such that $f(x) > \gamma$. Then the first term in (2) will be negative, the terms under summation will be zero, and the last term will be negative. Thus we have a contradiction.

In this example, we consider the 5-cycle, i.e., a graph with 5 nodes and 5 edges forming a closed chain. The number of cut is given by

\begin{gather*}
    f(x) = 2.5 - 0.5x_1x_2-0.5x_2x_3-0.5x_3x_4-0.5x_4x_5-0.5x_5x_1 \tag{3}
\end{gather*}

The SOSP is as follows.

Choose a fixed value of $\gamma$. For f(x) given in (3), find

\begin{align*}
    &\text{sum of squares } p_1(x) = \begin{bmatrix} 1\\x \end{bmatrix}^T Q \begin{bmatrix} 1\\x \end{bmatrix} \\
    &\text{polynomials } p_{i+1}(x) \text{ of degree 2, for } i = 1,\dots,n
\end{align*}

such that (2) is satisfied.

We choose $\gamma=4$. 

In [2]:
x0,x1,x2,x3,x4 = symbols("x0,x1,x2,x3,x4")
vartable = [x0,x1,x2,x3,x4]

# Number of cuts
f = 2.5 - 0.5*x0*x1 - 0.5*x1*x2 - 0.5*x2*x3 - 0.5*x3*x4 - 0.5*x4*x0

# Boolean constraints
bc = [None]*5
bc[0] = (x0**2-1)
bc[1] = (x1**2-1)
bc[2] = (x2**2-1)
bc[3] = (x3**2-1)
bc[4] = (x4**2-1)

# =============================================
# First, initialize the sum of squares program
prog = sosprogram(vartable)

# =============================================
# Then define SOSP variables

# -- p1(x) -- : sum of squares
# Monomial vector: 5 independent variables, degree <= 1
Z = monomials(vartable,[0,1])
p = [None]*6
prog,p[0] = sossosvar(prog,Z)

# -- p2(x) ... p6(x) : polynomials
# Monomial vector: 5 independent variables, degree <= 2
Z = monomials(vartable,range(3))
for i in range(5):
    prog,p[i+1] = sospolyvar(prog,Z)
    

# =============================================
# Next, define SOSP constraints

# Constraint : p1(x)*(gamma - f(x)) +  p2(x)*bc1(x)
#               + ... + p6(x)*bc5(x) - (gamma-f(x))^2 >= 0
gamma = 4

expr = p[0]*(gamma-f)
for i in range(1,6):
    expr = expr + p[i]*bc[i-1]
expr = expr - (gamma-f)**2

prog = sosineq(prog,expr)

# =============================================
# And call solver
options = {}
options['solver'] = 'scs'
options['eps_abs'] = 1e-9
options['eps_rel'] = 1e-9
prog = sossolve(prog,options,verbose=0)

Installed SDP solvers:  ['MOSEK', 'CVXOPT', 'SCS', 'SDPA']

 Residual norm 1.747211052594997e-11
cpusec: 0.02999
iter: 75
status: solved
pinf: 0.0
dinf: 0.0


The program is feasible, so $\gamma=4$ is indeed the maximum cut for 5-cycle.

### Citation:

[1]: A. Papachristodoulou, J. Anderson, G. Valmorbida, S. Prajna, P. Seiler, P. A. Parrilo, M. M. Peet, and D. Jagt, "4.6 MAX CUT," in _Sum of Squares Optimization Toolbox for MATLAB, User’s guide_, Version 4.00, 2021, pp. 43-48.

In [3]:
prog.solinfo['var']

{}